# Plot Line Graph of Daily Account Balance (sad)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, round

spark = SparkSession.builder.appName("Python Analytic Functions").getOrCreate()
 
text = spark.sparkContext.textFile("/home/jovyan/Resources/tx.csv")

txns = text.map(lambda st: st.split(",")).map(lambda el: (el[0], el[1], float(el[2]))).toDF(["date", "desc", "amount"])

# totalling amounts over dates
days = txns.groupBy("date").agg(sum("amount").alias("amount"))

# using Window Functions to accumulate balance
wind = Window.orderBy("date")
totl = days.withColumn("total", sum("amount").over(wind))
rnds = totl.select(totl["date"], round(totl["total"], 2).alias("total"))
coll = rnds.collect()

labels = list(map(lambda x: x[0], coll))
values = list(map(lambda x: x[1], coll))

# for graph to be legible we pick every 10th element
sel_labs = labels[::10]
sel_vals = values[::10]

import matplotlib.pyplot as plt
import numpy as np

# rotate x labels for legibility
plt.plot(sel_labs, sel_vals, color='red')
plt.xticks(np.arange(len(sel_labs)), sel_labs, rotation='vertical')
plt.xlabel('Date')
plt.ylabel('Balance')
plt.title('Daily Account Balance')
plt.grid(True)
plt.show()